In [3]:
from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
import pinecone
import os

In [8]:
from dotenv import load_dotenv


load_dotenv()
# Initialize Pinecone
pinecone.init(api_key=os.getenv('PINECONE_API_KEY'), environment='gcp-starter')

In [18]:
def initialize_pinecone_index():
    loader = OnlinePDFLoader("https://www.govinfo.gov/content/pkg/USCODE-2011-title18/pdf/USCODE-2011-title18.pdf")
    data = loader.load()
    print(f'You have {len(data)} document(s) in your data')
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(data)
    
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))
    index_name = 'lawyer'
    
    # Here we assume that Pinecone.from_texts is a valid method and that it returns something we can use later
    docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)
    
    return docsearchfrom langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

index_name = "langchain-chatbot"

index = docsearch

def get_similiar_docs(query,k=1,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

query = "what is a petty offense?"
similar_docs = get_similiar_docs(query)
similar_docs


In [19]:
# Running the QA chain
def run_qa_chain(query):
    llm = OpenAI(temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'))
    chain = load_qa_chain(llm, chain_type="stuff")  # Ensure 'stuff' is a valid chain_type
    docs = docsearch.similarity_search(query)
    answer = chain.run(input_documents=docs, question=query)
    return answer

In [2]:
# Set up your index on import of this module
docsearch = initialize_pinecone_index()

NameError: name 'initialize_pinecone_index' is not defined

In [9]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))
index_name = 'lawyer'

In [10]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [11]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

index_name = "langchain-chatbot"

index = docsearch

def get_similiar_docs(query,k=1,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

query = "what is a petty offense?"
similar_docs = get_similiar_docs(query)
similar_docs


[Document(page_content='Section, act June 25, 1948, ch. 645, 62 Stat. 686; Aug. 5, 1953, ch. 325, 67 Stat. 366; Pub. L. 87–845, § 3(a), Oct. 18, 1962, 76A Stat. 698; Pub. L. 90–357, § 59, June 22, 1968, 82 Stat. 248; Pub. L. 101–647, title XXXV, § 3519(c), Nov. 29, 1990, 104 Stat. 4923; Pub. L. 103–322, title XXXIII, § 330010(9), Sept. 13, 1994, 108 Stat. 2143, listed Title 18 sections applicable to and within Canal Zone.\n\n§ 19. Petty offense defined\n\nAs used in this title, the term ‘‘petty offense’’ means a Class B misdemeanor, a Class C mis- demeanor, or an infraction, for which the maxi- mum fine is no greater than the amount set forth for such an offense in section 3571(b)(6) or (7) in the case of an individual or section 3571(c)(6) or (7) in the case of an organization.\n\n(Added Pub. L. 100–185, § 4(a), Dec. 11, 1987, 101 Stat. 1279; amended Pub. L. 100–690, title VII, § 7089(a), Nov. 18, 1988, 102 Stat. 4409.)\n\n§ 15. Obligation or other security of foreign gov-\n\nernment 